In [12]:
from pyomo.environ import * 
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import folium 
from folium.plugins import AntPath


# Cambiar las rutas de los archivos
clients = pd.read_csv(r'case_2_cost\Clients.csv')  # Clientes
depots = pd.read_csv(r'case_2_cost\Depots.csv')    # Depósitos
distancias = pd.read_csv(r'Matrices/distancias2.csv')  # Matriz de distancias
tiempos = pd.read_csv(r'Matrices/tiempos2.csv')        # Matriz de tiempos
vehicles = pd.read_csv(r'case_2_cost\Vehicles.csv')    # Vehículos
costos_operativos = pd.read_csv(r'case_2_costs_vehicules\costos_operativos_combinados2.csv')   # Costos Operativos 



# Daily Maintenance [COP/day]
costos_mantenimiento = {
    'V1': 30000,
    'V2': 30000,
    'V3': 21000,
    'V4': 21000,
    'V5': 3000,
    'V6': 3000,
}

costos_mantenimiento_total = 108000

# Recharge/Fuel Cost [COP/(gal or kWh)]
costos_recarga = {
    'V1': 16000,
    'V2': 16000,
    'V3': 16000,
    'V4': 16000,
    'V5': 0,
    'V6': 0,
    'V7': 0,
    'V8': 0,
    'V9': 0,
    'V10': 220.73,
    'V11': 220.73,
    'V12': 220.73,
}



vehicles['ID'] = ['V' + str(i+1) for i in range(len(vehicles))]

# Crear conjuntos
distancias_dict = {(int(row["Desde"]+1), int(row["Hacia"]+1)): row["Distancia"] for _, row in distancias.iterrows()}


clientes = clients['LocationID'].tolist()  # IDs de los clientes
centros = depots['LocationID'].tolist()   # IDs de los depósitos
vehiculos = vehicles['ID'].tolist()  # IDs de los vehículos
productos = ['Product']  # Definimos tipos de productos (puedes ajustar según el problema)
nodos = centros + clientes  # Todos los nodos (clientes + depósitos)
clients.columns = clients.columns.str.strip()
depots.columns = depots.columns.str.strip()

# Crear parámetros
# 1. Demandas de clientes por producto (puede venir de Clients.csv o ser constante)
demandas_clients = {(row['LocationID'], p): row['Product'] for _, row in clients.iterrows() for p in productos}
demanda_centros = {
    (row['LocationID'], p): 0
    for _, row in depots.iterrows()
    for p in productos
}
demandas = {**demandas_clients, **demanda_centros}



# 2. Capacidades de los vehículos (Vehicles.csv)
capacidad_vehiculo = {row['ID']: row['Capacity'] for _, row in vehicles.iterrows()}

# 3. Rango máximo de los vehículos (Vehicles.csv)
rango_vehiculo = {row['ID']: row['Range']*1000 for _, row in vehicles.iterrows()}
print(rango_vehiculo)
# 4. Costo operativo 
costos_operativos_dict = {(row['Vehiculo'], row['Desde'], row['Hacia']): row['Costo'] 
                          for _, row in costos_operativos.iterrows()}



# 6. Capacidades de los centros de distribución (Depots.csv)
#capacidades_centro = {(row['DepotID'], p): row[f'Capacidad_{p}'] for _, row in depots.iterrows() for p in productos}

# 7. Distancias y tiempos (distancias1.csv y tiempos1.csv)
# Convertir archivos CSV a diccionarios

tiempos_dict = {(row['Desde'], row['Hacia']): row['Tiempo'] for _, row in tiempos.iterrows()}
# Crear diccionario de coordenadas para todos los nodos
all_coords = {
    **{row['LocationID']: (row['Latitude'], row['Longitude']) for _, row in clients.iterrows()},
    **{row['LocationID']: (row['Latitude'], row['Longitude']) for _, row in depots.iterrows()}
}


# Validar dimensiones
print(f"Clientes: {len(clientes)}, Centros: {len(centros)}, Vehículos: {len(vehiculos)}, Nodos: {len(nodos)}")





{'V1': 145852.07096486446, 'V2': 195690.8956922408, 'V3': 953172.608610164, 'V4': 1153486.9458305817, 'V5': 16627.680130757893, 'V6': 13602.810739289229}
Clientes: 30, Centros: 12, Vehículos: 6, Nodos: 42


In [13]:
class VRPModel:
    def __init__(self, clientes, centros, vehiculos, productos, distancias_dict, tiempos, demandas, capacidad_vehiculo, rango_vehiculo, costos_operativos_dict, costos_mantenimiento, costos_recarga,all_coords):
        """
        Inicializar parámetros del modelo VRP.
        """
        # Conjuntos y parámetros
        self.clientes = clientes
        self.centros = centros
        self.vehiculos = vehiculos
        self.productos = productos
        self.nodos = clientes + centros
        self.distancias_dict = distancias_dict
        self.tiempos = tiempos
        self.demandas = demandas
        self.capacidad_vehiculo = capacidad_vehiculo
        self.rango_vehiculo = rango_vehiculo
        self.costos_operativos_dict= costos_operativos_dict
        self.costos_mantenimiento = costos_mantenimiento
        self.costos_recarga = costos_recarga
        self.all_coords = all_coords

        
        
        # Crear modelo Pyomo
        self.model = ConcreteModel()

    def build_model(self):
        """
        Construir el modelo de optimización.
        """
        model = self.model

        # Sets
        model.C = Set(initialize=self.clientes)
        model.D = Set(initialize=self.centros)
        model.V = Set(initialize=self.vehiculos)
        model.P = Set(initialize=self.productos)
        model.N = Set(initialize=self.nodos) 

        # Variables
        model.x = Var(model.N, model.N, model.V, within=Binary)  # Ruta del vehículo
        model.z = Var(model.V, within=Binary)  # Activación del vehículo
        model.q = Var(model.N, model.P, model.V, within=NonNegativeReals)  # Carga transportada

        # Función objetivo: Minimizar costos
        def obj_expression(model):
            return sum(
                # Costos por distancia
                model.x[i, j, v] * costos_operativos_dict[(v, i, j)]
                for v in model.V for i in model.N for j in model.N if (v, i, j) in costos_operativos_dict) + costos_mantenimiento_total
            sum(
                # Costos de mantenimiento
                model.z[v] * self.costos_mantenimiento[v] for v in model.V)
            + sum(
                # Costos de recarga
                self.costos_recarga[v] * model.z[v] for v in model.V
            )
        model.obj = Objective(rule=obj_expression, sense=minimize)
        
        # Restricciones

        # 1. Cada cliente debe ser visitado exactamente una vez
        def restriccion_visita_unica(model, i):
            return sum(model.x[i, j, v] for j in model.N for v in model.V if i != j) == 1
        model.restriccion_visita_unica = Constraint(model.C, rule=restriccion_visita_unica)

        # 2. Conservación del flujo en los nodos
        def restriccion_flujo(model, i, v):
            return sum(model.x[i, j, v] for j in model.N if i != j) == sum(model.x[j, i, v] for j in model.N if i != j)
        model.restriccion_flujo = Constraint(model.N, model.V, rule=restriccion_flujo)

        # 3. Capacidad de los vehículos
        def restriccion_capacidad_vehiculo(model, v):
            return sum(model.q[i, p, v] for i in model.C for p in model.P) <= self.capacidad_vehiculo[v]
        model.restriccion_capacidad_vehiculo = Constraint(model.V, rule=restriccion_capacidad_vehiculo)

        #4. No viajes a si mismo
        def restriccion_no_viajes(model, i, v):
            return model.x[i, i, v] == 0
        model.restriccion_no_viajes = Constraint(model.N, model.V, rule=restriccion_no_viajes)
        
        #5. No viaaje de un deposito a otro deposito
        def restriccion_no_viajes_centros(model, i, j, v):
            return model.x[i, j, v] == 0
        model.restriccion_no_viajes_centros = Constraint(model.D, model.D, model.V, rule=restriccion_no_viajes_centros)
        
        # 5. Capacidad de los centros de distribución
        # def restriccion_capacidad_centro(model, k, p):
        #     return sum(model.q[i, p, v] for i in model.C for v in model.V) <= self.capacidades_centro[k, p]
        # model.restriccion_capacidad_centro = Constraint(model.D, model.P, rule=restriccion_capacidad_centro)

        # 6. Límite de rango de los vehículos
        def restriccion_rango_vehiculo(model, v):
            return sum(self.distancias_dict[(i, j)] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= self.rango_vehiculo[v]
        model.restriccion_rango_vehiculo = Constraint(model.V, rule=restriccion_rango_vehiculo)
        
        

        #7. Restricción de activación de vehículos
        M= len(self.nodos)
        def restriccion_activacion_vehiculo(model, v):
            return sum(model.x[i, j, v] for i in model.N for j in model.N if i != j) <= M * model.z[v]
        model.restriccion_activacion_vehiculo = Constraint(model.V, rule=restriccion_activacion_vehiculo)

        # 8. Eliminación de subtours (MTZ)
        model.u = Var(model.N, model.V, within=NonNegativeReals)
        def restriccion_subtours(model, i, j, v):
            if i != j:
                return model.u[i, v] - model.u[j, v] + len(model.N) * model.x[i, j, v] <= len(model.N) - 1
            return Constraint.Skip
        model.restriccion_subtours = Constraint(model.C, model.C, model.V, rule=restriccion_subtours)

        #9. Sale de un centro depot
        def restriccion_sale_vehiculo(model,v):
            return sum(model.x[d,j,v] for d in model.D for j in model.N)==1
        model.restriccion_sale_vehiculo= Constraint(model.V, rule= restriccion_sale_vehiculo)
        
        def restriccion_entra_vehiculo(model,v):
            return sum(model.x[i,d,v] for d in model.D for i in model.N)==1
        model.restriccion_entra_vehiculo= Constraint(model.V, rule= restriccion_entra_vehiculo)
        
        return model
    

    
    
    def solve_model(self):
        """
        Resolver el modelo utilizando HiGHS.
        """
        solver_name = "appsi_highs"
        solver = SolverFactory(solver_name)
        solver.options['parallel'] = 'on'
        solver.options['time_limit'] = 5*60  # 1-hour time limit
        solver.options['presolve'] = 'on'
        solver.options['mip_rel_gap'] = 0.01  # 1% relative gap
        solver.options['simplex_strategy'] = 1  # Dual simplex
        solver.options['simplex_max_concurrency'] = 8  # Max concurrency
        solver.options['mip_min_logging_interval'] = 10  # Log every 10 seconds
        solver.options['mip_heuristic_effort'] = 0.2  # Increase heuristic effort

        # Resolver el modelo
        result = solver.solve(self.model, tee=True)

        # Verificar el estado del solver
        if result.solver.termination_condition == TerminationCondition.infeasible:
            print("El modelo es infactible.")
        elif result.solver.termination_condition == TerminationCondition.optimal:
            print("Se encontró una solución óptima.")
            # Cargar la solución si es óptima
            self.model.solutions.load_from(result)
        elif result.solver.termination_condition == TerminationCondition.unbounded:
            print("El modelo es no acotado.")
        elif result.solver.termination_condition == TerminationCondition.maxTimeLimit:
            print("Se alcanzó el límite de tiempo.")
        else:
            print(f"Condición de terminación desconocida: {result.solver.termination_condition}")

        print(result)

        
    def display_results(self):
        print("\nRutas asignadas por vehículo:")
        for v in self.vehiculos:
            print(f"Vehículo: {v}")
            for i in self.nodos:
                for j in self.nodos:
                    if i != j and round(value(self.model.x[i, j, v])) == 1:  # Redondeamos el valor
                        print(f"   De {i} a {j}")
                        

    def visualize_routes_with_map(self):
        selected_routes = []
        for v in self.vehiculos:
            routes_for_v = [
                (i, j) for i in self.nodos for j in self.nodos
                if i != j and round(value(self.model.x[i, j, v])) == 1
            ]
            print(f"Rutas seleccionadas para el vehículo {v}: {routes_for_v}")
            selected_routes.append(routes_for_v)

        # Preparar colores y nodos
        colors = ['green', 'orange', 'darkblue', 'darkred', 'red', 'purple', 'black']
        node_colors = {}
        for k in range(len(self.vehiculos)):
            for route in selected_routes[k]:
                i, j = route
                idx_color = k % len(colors)
                if i not in node_colors and i > len(self.centros):
                    node_colors[i] = colors[idx_color]
                if j not in node_colors and j > len(self.centros):
                    node_colors[j] = colors[idx_color]

        # Crear el mapa
        mapa = folium.Map(location=all_coords[self.centros[0]], zoom_start=12)
        for idx in all_coords:
            lat, lon = all_coords[idx]
            icon, color = ("warehouse", 'gray') if idx in self.centros else ("envelope", node_colors.get(idx, "black"))

            folium.Marker(
                location=(lat, lon),
                popup=f"Nodo {idx}",
                icon=folium.Icon(color=color, icon=icon, prefix="fa"),
            ).add_to(mapa)

        for k in range(len(self.vehiculos)):
            for route in selected_routes[k]:
                i, j = route
                idx_color = k % len(colors)

                AntPath(
                    locations=[all_coords[i], all_coords[j]],
                    tooltip=f"Vehículo {self.vehiculos[k]}: {i} -> {j}",
                    color=colors[idx_color],
                ).add_to(mapa)

        # Guardar el mapa
        mapa.save("mapa_rutas_optimizadas.html")
        print("Mapa guardado como mapa_rutas_optimizadas.html")
        return mapa








In [14]:
# Instanciar la clase del modelo
vrp = VRPModel(
    clientes=clientes,
    centros=centros,
    vehiculos=vehiculos,
    productos=productos,
    distancias_dict=distancias_dict,
    tiempos=tiempos,
    demandas=demandas,
    capacidad_vehiculo=capacidad_vehiculo,
    rango_vehiculo=rango_vehiculo,
    costos_operativos_dict=costos_operativos_dict,
    costos_mantenimiento=costos_mantenimiento,
    costos_recarga=costos_recarga,
    all_coords=all_coords
)

# Construir el modelo
vrp.build_model()

# Resolver el modelo
vrp.solve_model()

vrp.display_results()

vrp.visualize_routes_with_map()

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+04]
  Cost   [8e+02, 2e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+06]
Presolving model
5526 rows, 8347 cols, 57588 nonzeros  0s
5027 rows, 7633 cols, 49430 nonzeros  9s

Solving MIP model with:
   5027 rows
   7633 cols (7453 binary, 0 integer, 0 implied int., 180 continuous)
   49430 nonzeros
MIP-Timing:          11 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. |

In [16]:
vrp.model.display()

Model unknown

  Variables:
    x : Size=10584, Index=N*N*V
        Key            : Lower : Value                  : Upper : Fixed : Stale : Domain
          (1, 1, 'V1') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 1, 'V2') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 1, 'V3') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 1, 'V4') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 1, 'V5') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 1, 'V6') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 2, 'V1') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 2, 'V2') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 2, 'V3') :     0 :                    0.0 :     1 : False : False : Binary
          (1, 2, 'V4') :     0 :                  